#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [22]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
replace ./weather_classification_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./weather_classification_data.csv  
replace ./data_scaled_labeled_check.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [23]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [25]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
1561,1.0,60,2.5,61.0,partly cloudy,995.83,1,Winter,2.5,mountain,Snowy
3778,10.0,60,14.5,78.0,overcast,1019.93,2,Spring,2.0,coastal,Rainy
2573,14.0,88,6.5,60.0,overcast,1016.66,0,Autumn,3.0,mountain,Rainy
9083,11.0,70,11.5,75.0,overcast,1013.41,1,Summer,2.5,coastal,Rainy
3804,40.0,61,9.0,4.0,clear,1017.38,11,Summer,8.5,mountain,Sunny


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [26]:
# START CODE HERE
object_columns = data.select_dtypes(include = 'object').columns.tolist()
non_object_columns = data.select_dtypes(exclude = 'object').columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [27]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [28]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer(transformers=[('obj', OrdinalEncoder(), object_columns),('num', StandardScaler(), non_object_columns)])

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [29]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [34]:
print(data_scaled_labeled)
X = data_scaled_labeled.drop(columns=["Weather Type"])
y = data['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state = 10)

       Cloud Cover  Season  Location  Weather Type  Temperature  Humidity  \
0              3.0     3.0       2.0           2.0    -1.042674 -0.431369   
1              2.0     1.0       0.0           1.0    -0.525006 -0.431369   
2              2.0     0.0       2.0           1.0    -0.294931  0.955217   
3              2.0     2.0       0.0           1.0    -0.467487  0.063841   
4              0.0     2.0       2.0           3.0     1.200553 -0.381848   
...            ...     ...       ...           ...          ...       ...   
13195          3.0     2.0       1.0           0.0     0.395293 -0.282806   
13196          3.0     1.0       1.0           1.0    -0.064857  0.113362   
13197          0.0     2.0       1.0           3.0     0.740404 -0.976099   
13198          3.0     0.0       0.0           0.0     0.337774  0.063841   
13199          3.0     0.0       0.0           1.0     0.107699  1.252343   

       Wind Speed  Precipitation (%)  Atmospheric Pressure  UV Index  \
0  

**TEST** (DON'T EDIT THIS CELL)

In [35]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [36]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,2.0,3.0,1.0,-1.617860,-0.233285,0.530916,1.106753,-0.222535,-0.779410,-0.730539
12278,2.0,3.0,1.0,-1.272748,1.252343,-0.265211,-0.114082,-0.489215,-0.779410,-0.433923
2859,3.0,1.0,0.0,-0.122375,0.162883,-0.771838,-0.708848,0.107858,-0.520104,-0.137308
5495,1.0,3.0,0.0,-1.675379,-1.916996,-0.048086,0.105042,4.062890,2.072951,-1.175461
8469,2.0,0.0,0.0,0.165218,0.261924,1.254668,1.638913,-0.422007,1.295035,-1.472077


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [37]:
# START CODE HERE

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=10)
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.21212121212122


**TEST** (DON'T EDIT THIS CELL)

In [38]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [ ]:
input = X_test.head()
input

In [ ]:
y = model.predict(input)
y

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
